<a href="https://colab.research.google.com/github/hjori66/Kaist-AI605-2021-Spring/blob/main/KAIST_AI605_Assignment_2_20194364.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIST AI605 Assignment 2: Token Classification with RNNs and Attention
Author: Minjoon Seo (minjoon@kaist.ac.kr)

TA in charge: Taehyung Kwon (taehyung.kwon@kaist.ac.kr)

**Due date**:  April 19 (Mon) 11:00pm, 2021  


Your name: Taehwan Kim

Your student ID: 20194364

Your collaborators: -

## Assignment Objectives
- Verify theoretically and empirically how Transformer's attention mechanism works for sequence modeling task.
- Implement Transformer's encoder attention layer from scratch using PyTorch.
- Design an Attention-based token classification model using PyTorch.
- Apply the token classification model to a popular machine reading comprehension task, Stanford Question Answering Dataset (SQuAD).
- (Bonus) Analyze pros and cons between using RNN + attention versus purely attention.

## Your Submission
Your submission will be a link to a Colab notebook that has all written answers and is fully executable. You will submit your assignment via KLMS. Use in-line LaTeX (see below) for mathematical expressions. Collaboration among students is allowed but it is not a group assignment so make sure your answer and code are your own. Also make sure to mention your collaborators in your assignment with their names and their student ids.

## Grading
The entire assignment is out of 100 points. There are two bonus questions with 30 points altogether. Your final score can be higher than 100 points.


## Environment
You will only use Python 3.7 and PyTorch 1.8, which is already available on Colab:

In [5]:
from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.10
torch 1.8.1+cu101


## 1. Transformer's Attention Layer

We will first start with going over a few concepts that you learned in your high school statistics class. The variance of a random variable $X$, $\text{Var}(X)$ is defined as $\text{E}[(X-\mu)^2]$ where $\mu$ is the mean of $X$. Furthermore, given two independent random variables $X$ and $Y$ and a constant $a$,
$$ \text{Var}(X+Y) = \text{Var}(X) + \text{Var}(Y), \quad \ldots \; \text{(1)}$$ 
$$ \text{Var}(aX) = a^2\text{Var}(X), \quad \ldots \; \text{(2)}$$
$$ \text{Var}(XY) = \text{E}(X^2)\text{E}(Y^2) - [\text{E}(X)]^2[\text{E}(Y)]^2. \quad \ldots \; \text{(3)}$$

**Problem 1.1** *(10 points)* Suppose we are given two sets of $n$ random variables, $X_1 \dots X_n$ and $Y_1 \dots Y_n$, where all of these $2n$ variables are mutually independent and have a mean of $0$ and a variance of $1$. Prove that
$$\text{Var}\left(\sum_i^n X_i Y_i\right) = n.$$

There is a typo in the formula $\text{(2)}$. I changed it.

**Answer 1.1** 

$$
\begin{align}
  \text{Var}\left(\sum_i^n X_i Y_i\right)
  &= \sum_i^n \text{Var} \left(X_i Y_i\right) \quad \because \text{(1), independence} \\
  &= \sum_i^n \left[\text{E}(X_i^2)\text{E}(Y_i^2) - [\text{E}(X_i)]^2[\text{E}(Y_i)]^2 \right] \quad \because \text{(3)} \\
  &= \sum_i^n \left[\text{E}((X_i-0)^2)\text{E}((Y_i-0)^2)\right] \\
  &= \sum_i^n \left[\text{E}((X_i-[\text{E}(X_i)])^2)\text{E}((Y_i-[\text{E}(Y_i)])^2)\right] \\
  &= \sum_i^n \left[\text{Var}(X_i) \text{Var}(Y_i)\right] \\
  &= \sum_i^n \left[1\right] = n \\
\end{align}
\\
$$

In Lecture 08 and 09, we discussed how the attention is computed in Transformer via the following equation,
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V.$$
**Problem 1.2** *(10 points)*  Suppose $Q$ and $K$ are matrices of independent variables each of which has a mean of $0$ and a variance of $1$. Using what you learned from Problem 1.1., show that
$$\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) = 1.$$

**Answer 1.2** 

$$
\begin{align}
  \text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right)
  &= \left(\frac{1}{\sqrt{d_k}}\right)^2 \text{Var}\left(QK^\top\right)  \quad \because \text{(2)} \\
  &= \frac{1}{d_k} \text{Var}\left(QK^\top\right) \\
\end{align}
\\
\\
$$

Then, we can focus on the one element of $QK$, $\left(QK\right)_{ij}$ without loss of generality.

$$
\begin{align}
  \frac{1}{d_k} \text{Var}\left(\left(QK\right)_{ij}^\top\right)
  &= \frac{1}{d_k} \text{Var}\left(\sum_{t}^{d_k} \left(Q_{it} K_{tj}\right) \right) \\
  &= \frac{1}{d_k} \left(\sum_{t}^{d_k} \text{Var}\left(Q_{it} K_{tj}\right) \right) \quad \because \text{(1), } Q_{it} \text{ and } Y_{tj} \text{ are mutually independent} \\
  &= \frac{1}{d_k} \left(d_k\right) = 1 \quad \because \text{Problem 1.1} \\
\end{align}
\\
$$

Therefore, 

$$
\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) = 1.
$$



**Problem 1.3** *(10 points)* What would happen if the assumption that the variance of $Q$ and $K$ is $1$ does not hold? Consider each case of it being higher and lower than $1$ and conjecture what it implies, respectively.

**Answer 1.3** \

If the variance of $Q_{ij}$ and $K_{ij}$ is higher than 1 for all i and j, then

$$
\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) > 1.
$$

Then, the variance of the output of the decoder becomes larger.
If we use the softmax function on this output, then the final result might be "too" sharp. (Actually, this is not true, because of the residual connection) \
So, I guess that the model overfits faster than original model.

\

Otherwse, if the variance of $Q_{ij}$ and $K_{ij}$ is lower than 1 for all i and j, then

$$
\text{Var}\left(\frac{QK^\top}{\sqrt{d_k}}\right) < 1.
$$

Then, the variance of the output of the decoder becomes smaller.
If we use the softmax function on this output, then the final result might be "too" smooth. 
\
So, I guess that the early training would be more unstable than normal although we use the bigger learning rate. The training time would be longer than the original version. 

## 2. Preprocessing SQuAD

We will use `datasets` package offered by Hugging Face, which allows us to easily download various language datasets, including Stanford Question Answering Dataset (SQuAD).

First, install the package:

In [6]:
!pip install datasets

     |████████████████████████████████| 194kB 8.9MB/s 
     |████████████████████████████████| 245kB 8.2MB/s 
     |████████████████████████████████| 112kB 14.8MB/s 


Then, download SQuAD and print the first example:

In [7]:
from datasets import load_dataset
squad_dataset = load_dataset('squad')
print(squad_dataset['train'][0])

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9. Subsequent calls will reuse this data.
{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']}, 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', 'id': '57

Here, `answer_start` corresponds to the character-level start position of the answer, and `text` is the answer text itself. You will note that `answer_start` and `text` fields are given as lists but they only contain one item each. In fact, you can safely assume that this is the case for the training data. During evaluation, however, you will utilize several possible answers so that your evaluation can be compared against all of them. So your code need to handle multiple-answers case as well.

As we discussed in Lecture 05, we want to formulate this task as a token classification problem. That is, we want to find which token of the context corresponds to the start position of the answer, and which corresponds to the end.

**Problem 2.1** *(10 points)* Write `preprocess()` function that takes a SQuAD example as the input and outputs space-tokenized context and question, as well as the start and end token position of the answer if it has the answer field. That is, a pseudo code would look like:
```python
def preprocess(example):
  out = {'context': ['each', 'token'], 
         'question': ['each', 'token']}
  if 'answers' not in example:
    return out
  out['answers'] = [{'start': 3, 'end': 5}]
  return out
```
Verify that this code works by comparing between the original answer text and the concatenation of the answer tokens from start to end in training data. Report the percentage of the questions that have exact match.

**Answer 2.1**



In [8]:
def preprocess(example):
  def tokenizer(sentence):
    if sentence is None:
      return list()
    return sentence.split()
  out = dict()

  answer = example['answers']['text'][0]
  answer_start = example['answers']['answer_start'][0]
  answer_end = answer_start + len(answer)
  context = example['context']
  question = example['question']

  out['context'] = tokenizer(context)
  out['question'] = tokenizer(question)

  if (answer_start == 0 or context[answer_start-1] == ' ') \
    and (answer_end == len(context) or context[answer_end] == ' '):
    n_tokens_before_answer = len(tokenizer(context[:answer_start]))
    n_tokens_answer = len(tokenizer(answer))
    out['answers'] = [{'start': n_tokens_before_answer, 'end': n_tokens_before_answer+n_tokens_answer}]

  return out

n_exact_match = 0.0
for i, example in enumerate(squad_dataset['train']):
  out = preprocess(example)
  if 'answers' in out.keys():
    n_exact_match += 1

print("number of answers that have exact match : ", n_exact_match)
print("number of training data : ", len(squad_dataset['train']))
print("the percentage of the exact match : ", n_exact_match / len(squad_dataset['train']))


number of answers that have exact match :  45983.0
number of training data :  87599
the percentage of the exact match :  0.5249260836310917


In [ ]:
number of answers that have exact match :  45983.0
number of training data :  87599
the percentage of the exact match :  0.5249260836310917 -> pretty low

We want to maximize the percentage of the exact match. You might see a low percentage however, due to bad tokenization. For instance, such space-based tokenization will fail to separate between "world" and "!" in "hello world!". 

**Problem 2.2** *(10 points)* Write an advanced tokenization model that always separates non-alphabet characters as independent tokens. For instance, "hello1 world!!" will be tokenized into "hello", "1", "world", "!", and "!". Using this new tokenizer, re-run the `preprocess` function and report the exact match percentage. How does the ratio change?

**Answer 2.2**



In [22]:
def find_nonalpha_list(dataset):
  nonalpha_list = list()
  for example in dataset:
    for c in example['context']:
      if not c.isalpha() and c not in nonalpha_list:
        nonalpha_list.append(c)
    for c in example['question']:
      if not c.isalpha() and c not in nonalpha_list:
        nonalpha_list.append(c)
  return nonalpha_list


def preprocess(example, nonalpha_list):
  def tokenizer(sentence, nonalpha_list):
    if sentence is None:
      return list()

    for nonalpha_token in nonalpha_list:
      sentence = sentence.replace(nonalpha_token, ' ' + nonalpha_token + ' ')

    sentence = ' '.join(sentence.split())
    return sentence.split()
  out = dict()

  answer = example['answers']['text'][0]
  answer_start = example['answers']['answer_start'][0]
  answer_end = answer_start + len(answer)
  context = example['context']
  question = example['question']

  out['context'] = tokenizer(context, nonalpha_list)
  out['question'] = tokenizer(question, nonalpha_list)

  if (answer_start == 0 or context[answer_start-1] in nonalpha_list) \
    and (answer_end == len(context) or context[answer_end] in nonalpha_list):
    n_tokens_before_answer = len(tokenizer(context[:answer_start], nonalpha_list))
    n_tokens_answer = len(tokenizer(answer, nonalpha_list))
    out['answers'] = [{'start': n_tokens_before_answer, 'end': n_tokens_before_answer+n_tokens_answer}]

  return out

train_dataset = squad_dataset['train']
nonalpha_list = find_nonalpha_list(train_dataset)
print("nonalpha_list : ", nonalpha_list)

n_exact_match = 0.0
for i, example in enumerate(squad_dataset['train']):
  out = preprocess(example, nonalpha_list)
  if 'answers' in out.keys():
    n_exact_match += 1

print("number of answers that have exact match : ", n_exact_match)
print("number of training data : ", len(squad_dataset['train']))
print("the percentage of the exact match : ", n_exact_match / len(squad_dataset['train']))


nonalpha_list :  [',', ' ', '.', "'", '"', '1', '8', '5', '(', '3', ')', '?', '-', '7', '6', '9', '2', '0', ';', '–', '&', '4', '%', '$', '[', ']', '/', ':', '#', '—', '!', '“', '’', '”', '<', '\u200b', '̃', '£', '½', '+', '¢', '−', '°', '>', '€', '《', '》', '±', '~', '¥', '²', '❤', '=', '\u200e', '͡', '́', '`', '्', 'ु', 'ः', 'ॊ', 'ि', 'ा', '\u200d', '\u200c', '*', '‘', '\u3000', '•', '§', '⁄', '\n', '̯', '̩', '…', '·', 'ָ', 'ִ', 'ׁ', 'ַ', 'ּ', 'ְ', 'ّ', '⟨', '◌', '⟩', '˭', '̤', '♠', '∅', '̞', '×', '̥', '′', '″', '\ufeff', '_', 'ֿ', '´', '^', '̧', '̄', '→', '‑', '，', '₹', '\u202f', '♯', '₂', '₥', '⁊', '\u2009', '{', '}', '|', '@', '̪', '‚', '›', 'ׂ', 'ֵ', 'ِ', 'ْ', 'َ', '̍', '˥', '˨', '˩', '¡', '√', '¿', 'ာ', 'း', 'ُ', '≥', '˚', '≈', '⋅', 'ี', '︘', '�', '～', '〜', '̀', 'ོ', '་', '˧', 'ಾ', 'ು', '್', 'া', '্', 'ಿ', '∗', '∈', '≡', '∖', '№', '÷', 'ٔ', '¶', 'ิ', '₤', '♆', '⅓', '∝', '¼', 'ٍ', 'ֹ', '̌', '。', '̠', '₯']
number of answers that have exact match :  87108.0
number of training data :

In [ ]:
nonalpha_list :  [',', ' ', '.', "'", '"', '1', '8', '5', '(', '3', ')', '?', '-', '7', '6', '9', '2', '0', ';', '–', '&', '4', '%', '$', '[', ']', '/', ':', '#', '—', '!', '“', '’', '”', '<', '\u200b', '̃', '£', '½', '+', '¢', '−', '°', '>', '€', '《', '》', '±', '~', '¥', '²', '❤', '=', '\u200e', '͡', '́', '`', '्', 'ु', 'ः', 'ॊ', 'ि', 'ा', '\u200d', '\u200c', '*', '‘', '\u3000', '•', '§', '⁄', '\n', '̯', '̩', '…', '·', 'ָ', 'ִ', 'ׁ', 'ַ', 'ּ', 'ְ', 'ّ', '⟨', '◌', '⟩', '˭', '̤', '♠', '∅', '̞', '×', '̥', '′', '″', '\ufeff', '_', 'ֿ', '´', '^', '̧', '̄', '→', '‑', '，', '₹', '\u202f', '♯', '₂', '₥', '⁊', '\u2009', '{', '}', '|', '@', '̪', '‚', '›', 'ׂ', 'ֵ', 'ِ', 'ْ', 'َ', '̍', '˥', '˨', '˩', '¡', '√', '¿', 'ာ', 'း', 'ُ', '≥', '˚', '≈', '⋅', 'ี', '︘', '�', '～', '〜', '̀', 'ོ', '་', '˧', 'ಾ', 'ು', '್', 'া', '্', 'ಿ', '∗', '∈', '≡', '∖', '№', '÷', 'ٔ', '¶', 'ิ', '₤', '♆', '⅓', '∝', '¼', 'ٍ', 'ֹ', '̌', '。', '̠', '₯']
number of answers that have exact match :  87108.0
number of training data :  87599
the percentage of the exact match :  0.9943949131839405 -> now, it is OK

## 3. LSTM Baseline for SQuAD

We will bring and reuse our model from Assignment 1. There are two key differences, however. First, we need to classify each token instead of the entire sentence. Second, we have two inputs (context and question) instead of just one. 

Before resolving these differences, you will need to define your evaluation function to correctly evaluate how well your model is doing. Note that the evaluation was very straightforward in Assignment 1's sentiment classification (it is either positive or negative) while it is a bit complicated in SQuAD. We will use the evaluation function provided by `datasets`. You can access to it via the following code.  

In [23]:
from datasets import load_metric
squad_metric = load_metric('squad')

You can also easily learn about how to use the function by simply typing the function:

In [24]:
squad_metric

Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   

**Problem 3.1** *(10 points)* Let's resolve the first issue here. Hence, for now, assume that your only input is context and you want to obtain the answer without seeing the question. While this may seem to be a non-sense, actually it can be considered as modeling the prior $\text{Prob}(a|c)$ before observing $q$ (we ultimately want $\text{Prob}(a|q,c)$). Transform your model into a token classification model by imposing $\text{softmax}$ over the tokens instead of predefined classes. You will need to do this twice for each of start and end. Report the accuracy (using the metric above) on `squad_dataset['validation']`. 

**Answer 3.1**



In [ ]:
.

**Problem 3.2** *(10 points)*  Now let's resolve the second issue, by simply concatenating the two inputs into one sequence. The simplest way would be to append the the question at the start *OR* the end of the context. If you put it at the start, you will need to shift the start and the end positions of the answer accordingly. If you put it at the end, it will be necesary to use bidirectional LSTM for the context to be aware of what is ahead (though it is recommended to use bidirectional LSTM even if the question is appended at the start). Whichever you choose, carry it out and report the accuracy. How does it differ from 3.1?

**Answer 3.2**



In [ ]:
.

## 4. LSTM + Attention for SQuAD

**Problem 4.1** *(20 points)* Here, we will be appending an attention layer on top of LSTM outputs. We will use a single-head attention sublayer from Transformer. That is, you will implement 
$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d}}\right)V,$$
where $Q, K, V$ is obtained by the linear transformation of the hidden states of the LSTM outputs $H$, i.e. $Q = HW^Q, K=HW^K, V=HW^V$ ($W^Q, W^K, W^V \in \mathbb{R}^{d \times d}$ are trainable weights). Note that the output of $\text{Attention}$ layer has the same dimension as $H$, so you can directly append your token classification layer on top of it. Report the accuracy and compare it with 3.2.



**Answer 4.1**



In [ ]:
.

**Problem 4.2** *(10 points)* On top of the attention layer, let's add another layer of (bi-directional) LSTM. So this will look like a *sandwich* where the LSTM is bread and the attention is ham. How does it affect the accuracy? Explain why do you think this happens. 

**Answer 4.2**



In [ ]:
.

## 5. Attention is All You Need

**Problem 5.1 (bonus)** *(20 points)*  Implement full Transformer encoder to entirely replace LSTMs. You are allowed to copy and paste code from [*Annotated Transformer*](https://nlp.seas.harvard.edu/2018/04/03/attention.html) (but nowhere else). Report the accuracy and explain what seems to happening with attetion-only model compared to LSTM+Attention model(s). 

**Problem 5.2 (bonus)** *(10 points)* Replace Transformer's sinusoidal position encoding with a fixed-length (of 256) position embedding. That is, you will create a 256-by-$d$ trainable parameter matrix for the position encoding that replaces the variable-length sinusoidal encoding. What is the clear disdvantage of this approach? Report the accuracy and compare it with 5.1. Note that this also has a clear advantage, as we will see in our future lecture on Pretrained Language Model, and more specifically, BERT (Devlin et al., 2018).

**Answer 5.1**

